In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser


In [6]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash")


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://blog.langchain.dev/langgraph-platform-ga/")

docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [10]:


from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter =  RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)



In [11]:
from langchain_community.vectorstores import FAISS

In [12]:
vector = FAISS.from_documents(documents, embeddings)

In [13]:
retriever = vector.as_retriever()

In [14]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look uo up in to get information relevant to the conversation.")
    ]
)

In [15]:
from langchain.chains import create_history_aware_retriever

retriever_chain = create_history_aware_retriever(
    model, retriever, prompt
)

In [16]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!")
]

In [17]:
retriever_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how",
    }
    
)

[Document(id='8dbc9c8f-e387-4e91-ad54-4bcaf5521aeb', metadata={'source': 'https://blog.langchain.dev/langgraph-platform-ga/', 'title': 'LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents', 'description': 'LangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy', 'language': 'en'}, page_content='management console makes it easier to enforce consistency, monitor behavior, and ship updates safely — all without needing to re-deploy or touch code every time. You can:Discover available agents in the agent registryCreate different versions of your agent (“assistants”) in LangGraph platform, allowing you to reuse common agent architectures\xa0Leverage other agents as “Remote Graphs”, allowing you to create multi agent architectures that run in a distributed mannerFor companies like Qualtrics, centralizing agent management with LangGraph Platform has been critical to driving effi

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's questions based on the below context. \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ]
)

In [19]:
# Bu metod dokümanları modele geçirebilmek için bir chain oluşturmak için
from langchain.chains.combine_documents import create_stuff_documents_chain 

documents_chain = create_stuff_documents_chain(model, prompt)



In [20]:
from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever_chain, documents_chain)

In [21]:
# Testing Conversational RAG
chat_history = [
    HumanMessage(content="Can I use LangGraph for agent runtimes?"),
    AIMessage(content="Yes!"),
]

In [22]:
response = retriever_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how"
    }
)

In [23]:
print(response)

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})], 'input': 'Tell me how', 'context': [Document(id='8dbc9c8f-e387-4e91-ad54-4bcaf5521aeb', metadata={'source': 'https://blog.langchain.dev/langgraph-platform-ga/', 'title': 'LangGraph Platform is now Generally Available: Deploy & manage long-running, stateful Agents', 'description': 'LangGraph Platform, our infrastructure for deploying and managing agents at scale, is now generally available. Learn how to deploy', 'language': 'en'}, page_content='management console makes it easier to enforce consistency, monitor behavior, and ship updates safely — all without needing to re-deploy or touch code every time. You can:Discover available agents in the agent registryCreate different versions of your agent (“assistants”) in LangGraph platform, allowing you to reuse common agent architectures\xa0Levera

In [24]:
print(response['answer'])

LangGraph Platform is designed specifically for deploying and managing long-running, stateful agents.  You can use it in several ways, depending on your needs and technical capabilities:


**1.  Direct Deployment:**  The easiest way is via 1-click deployment from the management console, using a native GitHub integration.  You select your repository, and LangGraph handles the rest.  This accelerates agent development.

**2. API Integration:** For more customized workflows, LangGraph provides 30 API endpoints to design user experiences tailored to your interaction patterns.  This gives you fine-grained control over the deployment and management of your agents.

**3. LangGraph Studio:** This built-in IDE helps you visualize and debug agent workflows in real time.  You can monitor agent trajectories, handle branching logic and retries, inspect memory/state, and easily identify and fix issues.  This visual approach makes development and troubleshooting much more efficient.

**4.  Pre-built 